In [34]:
%matplotlib inline
from matplotlib import pyplot as plt
from glob import glob
import numpy as np
import pandas as pd
import os
import sys
import random


from PIL import Image
import skvideo.io
import skvideo.datasets
import json
from tqdm import tqdm

import torch
import torchvision
from torchvision import transforms


sys.path.append('..')
from scripts.mobilenet import mnv2, pretrained_model_fnames

In [58]:
videos = glob('/MediaArchivePool/datasets/video/Moments_In_Time/Moments_in_Time_Raw/validation/[a-h]*/*')
#videos

In [43]:
N_SET = 5
b_dict = {}
for i in range(len(videos)):
    metadata = skvideo.io.ffprobe(videos[i])
    length = int(metadata['video']['@nb_frames'])
    #print(videos[i], length)
    boundaries = sorted(random.sample(range(length), N_SET-1)) + [length]    
    for i in range(1, len(boundaries)):
        boundaries[i] = boundaries[i] - boundaries[i-1]
    b_dict[videos[i]] = boundaries

#b_dict

In [44]:
# This normalize is specific to ImageNet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

input_size = 224
test_trans = transforms.Compose([
    transforms.Resize(int(input_size/0.875)),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    normalize,
    ])

In [4]:
pretrained_model_fnames

{'mnv1': '../data/mnv1/mobilenet_sgd_rmsprop_69.526.tar',
 'mnv2': '../data/mnv2/mobilenet_v2.pth.tar'}

In [5]:
model = mnv2(pretrained=True, freeze=False) #look at ../scripts/mobilenet.py to see additional args

Pretrained fname ../data/mnv2/mobilenet_v2.pth.tar


In [83]:
def compute_features(transform, model, videofile, n_set=5):
    vid_in = skvideo.io.FFmpegReader(videofile)
    set_vectors = []
    if torch.cuda.is_available():
        model.cuda()
    for i in range(n_set):
        predictions = []
        cpt = 0
        frames_iterator = tqdm(vid_in)
        #for idx, frame in  enumerate(tqdm(vid_in)):
        while cpt < boundaries[i]:
            input_frame = transform(Image.fromarray(frame))
            with torch.no_grad():
                input_img_var = torch.autograd.Variable(input_frame)
            if torch.cuda.is_available():
                input_img_var = input_img_var.cuda()

            # compute output
            output = model(input_img_var.unsqueeze(0))
            # this assumes that the model has two outputs 
            # ['emb'] and ['prob'] and we want the first output
            if torch.cuda.is_available():
                output = output[0].cpu().data.numpy()

            # check that output has same dimensionality as expected labels
            #assert output.shape[1] == len(all_classes)
            predictions.append(output)
            cpt += 1

        vidout = np.array(predictions) 
        vidout = vidout.mean(axis=0)
        #print(vidout.shape)
        set_vectors.append(vidout)
        vid_in.close()
        
    #This is the random order in which we shuffle the "blocks" of the video
    #So we need to figure out the inverse permutation function that is going to serve as the correct order
    random_order = random.sample(range(n_set), n_set)
    y = np.zeros(n_set, dtype=int)
    print(random_order, len(set_vectors))
    for k, v in enumerate(random_order):
        y[v] = k
    # we reorder the feature representation of the blocks now that we have computed the the correct order from the 
    #shuffled one
    set_vectors = [set_vectors[x] for x in random_order]
    X = np.stack(set_vectors, axis=0)
        
    return X, y



#X, y = compute_features(test_trans, model, videos[0])
#X.shape

In [84]:
n_videos = len(videos)
n_train = int(n_videos*.7)
n_val = int(n_videos*.15)

train_videos = random.sample(videos, n_train)
not_train = [x for x in videos if x not in train_videos]
val_videos = random.sample(not_train, n_val)
test_videos = [x for x in not_train if x not in not_train]

In [85]:
data_dict = {'train': [], 'test': [], 'val': []}
for i, vid in enumerate(train_videos):
    X, y = compute_features(test_trans, model, train_videos[i], n_set=N_SET)
    data_dict['train'].append((X, y))
    
for i, vid in enumerate(val_videos):
    X, y = compute_features(test_trans, model, val_videos[i], n_set=N_SET)
    data_dict['val'].append((X, y))
    
for i, vid in enumerate(test_videos):
    X, y = compute_features(test_trans, model, test_videos[i], n_set=N_SET)
    data_dict['test'].append((X, y))











0it [00:00, ?it/s]










0it [00:00, ?it/s]




















0it [00:00, ?it/s]




















0it [00:00, ?it/s]




















0it [00:00, ?it/s]





























0it [00:00, ?it/s]

[1, 3, 4, 0, 2] 5













0it [00:00, ?it/s]




















0it [00:00, ?it/s]




















0it [00:00, ?it/s]




















0it [00:00, ?it/s]





















[2, 1, 4, 0, 3] 5












0it [00:00, ?it/s]










0it [00:00, ?it/s]




















0it [00:00, ?it/s]




















0it [00:00, ?it/s]




















0it [00:00, ?it/s]





















...































0it [00:00, ?it/s]






























0it [00:00, ?it/s]




























































0it [00:00, ?it/s]




























































0it [00:00, ?it/s]




























































0it [00:00, ?it/s]





























































[1, 2, 0, 4, 3] 5
































0it [00:00, ?it/s]






























0it [00:00, ?it/s]




























































0it [00:00, ?it/s]































0it [00:00, ?it/s]






























































0it [00:00, ?it/s]































































[1, 3, 0, 2, 4] 5


BlockingIOError: [Errno 11] Resource temporarily unavailable